In [12]:
import pandas as pd
from konlpy.tag import Okt
import re
from kss import split_sentences
# --- 1. 데이터 불러오기 ---
# (이전과 동일한 안전한 로딩 함수 사용)
def load_csv_robustly(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        return df
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='cp949')
        return df

cheongju = load_csv_robustly("./crawled_data/merged_cheongju.csv")
other = load_csv_robustly("./crawled_data/merged_other.csv")

COLUMN_TO_ANALYZE = 'content_cleaned' # 분석할 열 이름

# --- 2. 분석 함수 정의 및 실행 ---
# (이전 단계에서 제안드린 카테고리별 문장 추출 함수)
okt = Okt()
category_keywords = {
    '음식': ['맛', '음식', '메뉴', '요리', '신선', '재료', '소스', '양', '식감', '퀄리티', '고기', '파스타', '피자', '초밥', '국물', '육수', '반찬', '디저트', '커피', '음료', '짜다', '싱겁다', '달다'],
    '분위기': ['분위기', '인테리어', '디자인', '느낌', '뷰', '전망', '공간', '자리', '좌석', '조명', '음악', '데이트', '모임', '깔끔', '깨끗', '웨이팅', '대기'],
    '가격': ['가격', '가성비', '비용', '금액', '저렴', '비싸', '할인', '무료', '추가', '계산', '가심비', '만원'],
    '서비스': ['서비스', '직원', '사장님', '친절', '응대', '주차', '예약', '위치', '화장실', '리필', '주문', '서빙', '불만족']
}

def extract_sentences_by_category(text):
    text = str(text) # 숫자 등 다른 타입이 들어올 경우를 대비
    category_sentences = {'음식_문장': [], '분위기_문장': [], '가격_문장': [], '서비스_문장': []}
    cleaned_text = re.sub(r"[^0-9가-힣?.!,¿]+", " ", text)
    sentences = re.split(r'[.!?]\s+|\n', cleaned_text)
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence: continue
        try:
            morphs = okt.pos(sentence, stem=True)
            words = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']]
        except:
            words = []
        for category, keywords in category_keywords.items():
            if any(keyword in words for keyword in keywords):
                category_sentences[f'{category}_문장'].append(sentence)
    return category_sentences

# --- 분석 실행 및 결과 병합 ---
print("분석 시작...")
# cheongju 데이터프레임 분석
sentence_results_cj = cheongju[COLUMN_TO_ANALYZE].apply(extract_sentences_by_category)
sentence_df_cj = pd.json_normalize(sentence_results_cj)
cheongju_final = pd.concat([cheongju, sentence_df_cj], axis=1)

print("청주 데이터 분석 완료.")

# other 데이터프레임 분석
sentence_results_ot = other[COLUMN_TO_ANALYZE].apply(extract_sentences_by_category)
sentence_df_ot = pd.json_normalize(sentence_results_ot)
other_final = pd.concat([other, sentence_df_ot], axis=1)
print("분석 완료.")
print("="*50)



# --- 3. 분석 결과가 추가된 데이터프레임을 새로운 CSV 파일로 저장 ---

# 저장할 새로운 파일 경로 정의
output_cheongju_path = "./crawled_data/merged_cheongju_analyzed.csv"
output_other_path = "./crawled_data/merged_other_analyzed.csv"

print("새로운 CSV 파일로 저장 시작...")

# cheongju_final 데이터프레임 저장
cheongju_final.to_csv(output_cheongju_path, encoding='utf-8-sig', index=False)
print(f"청주 데이터 분석 결과가 '{output_cheongju_path}' 파일에 저장되었습니다.")

# other_final 데이터프레임 저장
other_final.to_csv(output_other_path, encoding='utf-8-sig', index=False)
print(f"다른 지역 데이터 분석 결과가 '{output_other_path}' 파일에 저장되었습니다.")
print("="*50)

분석 시작...
청주 데이터 분석 완료.
분석 완료.
새로운 CSV 파일로 저장 시작...
청주 데이터 분석 결과가 './crawled_data/merged_cheongju_analyzed.csv' 파일에 저장되었습니다.
다른 지역 데이터 분석 결과가 './crawled_data/merged_other_analyzed.csv' 파일에 저장되었습니다.


In [13]:
print(cheongju_final.head())  # 청주 데이터프레임 상위 5개 행 출력
print(other_final.head())  # 다른 지역 데이터프레임 상위 5개 행 출력

                                   title  \
0           청주 가경동 더덕한상 점심특선 가성비 식당 솔직후기   
1       청주 가경동 냉면 식당 8.000원이고 뜨거운 육수 나온다   
2               청주 가경동 맛집 가경터미널 짜글이 진주식당   
3  청주 가경동 일식 코스 요리 추천 가족 모임하기 좋은 룸식당...    
4                  청주 가경동맛집 강서동맛집금성식당 후기   

                                             content        date  \
0  저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔는...  2025-04-01   
1  ​청주 가경동 냉면 식당 다녀왔어요출장을 청주로 가면 가경동에서 손님을 만나다 보니...  2025-07-28   
2  50m© NAVER Corp.더보기/OpenStreetMap지도 데이터x© NAVE...  2023-07-26   
3  청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 @네정​​​...  2025-07-13   
4  ​​​며칠 전 친구랑 청주 금성식당 다녀왔어요. 원래 고깃집은 워낙 자주 가는 편인...  2025-05-26   

                                     content_cleaned  \
0  저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔는...   
1  ​청주 가경동 냉면 식당 다녀왔어요출장을 청주로 가면 가경동에서 손님을 만나다 보니...   
2  50m.  NAVER Corp.더보기/OpenStreetMap지도 데이터x.  NA...   
3  청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 @네정​​​...   
4  ​​​며칠 전 친구랑 청주 금성식당 다녀왔어요. 원래 고깃집은 워낙 자주 가는

In [15]:

cheongju_final.head(10)

,title,content,date,content_cleaned,음식_문장,분위기_문장,가격_문장,서비스_문장
0,청주 가경동 더덕한상 점심특선 가성비 식당 솔직후기,저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔는...,2025-04-01,저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔는...,"[지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가더덕한상충청북도 청주시 흥덕...","[지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가더덕한상충청북도 청주시 흥덕...","[지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가더덕한상충청북도 청주시 흥덕...",[저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔...
1,청주 가경동 냉면 식당 8.000원이고 뜨거운 육수 나온다,​청주 가경동 냉면 식당 다녀왔어요출장을 청주로 가면 가경동에서 손님을 만나다 보니...,2025-07-28,​청주 가경동 냉면 식당 다녀왔어요출장을 청주로 가면 가경동에서 손님을 만나다 보니...,[날씨가 덥다 보니 사람들이 많이 먹는 음식 중에 하나가 냉면일 거 같아요저도 들어...,[날씨가 덥다 보니 사람들이 많이 먹는 음식 중에 하나가 냉면일 거 같아요저도 들어...,[날씨가 덥다 보니 사람들이 많이 먹는 음식 중에 하나가 냉면일 거 같아요저도 들어...,[날씨가 덥다 보니 사람들이 많이 먹는 음식 중에 하나가 냉면일 거 같아요저도 들어...
2,청주 가경동 맛집 가경터미널 짜글이 진주식당,50m© NAVER Corp.더보기/OpenStreetMap지도 데이터x© NAVE...,2023-07-26,50m. NAVER Corp.더보기/OpenStreetMap지도 데이터x. NA...,"[지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가짜글이전문점 진주식당충청북도...","[매장 둘러보기 여름에도 시원하고 쾌적한 공간 테이블 많았고, 매장 전반적으로 깔끔...",[메뉴 주문 후 이벤트 참여한다고 말씀해 주시면 바로 계산하면서 영수증 갖다주시니까...,[메뉴 2인 이상 주문시 부침개 서비스 대추나무집과 비슷하게 짜글이가 두종류로 구분...
3,청주 가경동 일식 코스 요리 추천 가족 모임하기 좋은 룸식당...,"청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 @네정​​​...",2025-07-13,"청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 @네정​​​...","[청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 네정 안녕...","[청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 네정 안녕...",[메뉴 구성가경동 가족 모임하기 좋은 룸식당 기본 코스와 고민하다가편백찜과 해산물 ...,"[기념일인 만큼 분위기 좋고,조용하게 식사할 수 있는 룸식당프라이빗한 코스요리 맛집..."
4,청주 가경동맛집 강서동맛집금성식당 후기,​​​며칠 전 친구랑 청주 금성식당 다녀왔어요. 원래 고깃집은 워낙 자주 가는 편인...,2025-05-26,​​​며칠 전 친구랑 청주 금성식당 다녀왔어요. 원래 고깃집은 워낙 자주 가는 편인...,"[원래 고깃집은 워낙 자주 가는 편인데요, 친구가 요즘 청주에서 먹은 고기 중 최고...","[생갈비도 마블링이 예쁘게 들어가 있어서 구울 때부터 고소한 향이 올라오는데, 입에...",[고 할 정도로 맛있게 먹고 온 곳이라 이렇게 후기로 남겨보려고 해요.저희가 주문한...,[고 할 정도로 맛있게 먹고 온 곳이라 이렇게 후기로 남겨보려고 해요.저희가 주문한...
5,청주 가경동 강서동 삼겹살 맛집 금성식당 둘이서 든든하게...,금성식당주소｜충북 청주시 흥덕구 강서동 81-15전화번호｜0507-1400-7597...,2025-05-07,금성식당주소｜충북 청주시 흥덕구 강서동 81-15전화번호｜0영업시간｜12:00 - ...,[위치는 터미널에서 걸어서 10분이라터미널 근처 맛집 찾는다면 권해드려요 주차는 따...,[싶었답니다 사장님의 진심이 한가득 느껴졌어요 저는 김치찌개를 골라주었지만사실 3가...,[칼집을 내서 만든 독특한칼메기 180 전부 다 맛나보여서 고르기 어렵길래바로 사장...,[금성식당주소 충북 청주시 흥덕구 강서동 81 15전화번호 0영업시간 12 00 2...
6,[청주 가경동 맛집] 파스타가 맛있는 믹스레스토랑,"[청주 가경동 맛집] 파스타가 맛있는 믹스레스토랑안녕하세요, 비비앙뜨에요. 며칠 전...",2025-01-17,"[청주 가경동 맛집] 파스타가 맛있는 믹스레스토랑안녕하세요, 비비앙뜨에요. 며칠 전...","[청주 가경동 맛집 파스타가 맛있는 믹스레스토랑안녕하세요, 비비앙뜨에요, 그래서 저...","[다들 솔로일 때 부터 이어진 모임이 유부녀가 되고, 출산까지 한 엄마들의 모임으로...","[그래서 저희가 맛도 맛이지만 분위기, 가격까지 세가지를 모두 아우르는 청주 가경동...","[지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가믹스레스토랑 본점충청북도 청..."
7,청주 가경동맛집 &quot;금성식당&quot; 생갈비 밀면 맛집,안녕하세요 또먹었서연입니다💎​오늘은 가경동맛집 생갈비를 잘하는금성식당에서 저녁식사하...,2025-08-03,안녕하세요 또먹었서연입니다. ​오늘은 가경동맛집 생갈비를 잘하는금성식당에서 저녁식사...,"[0 예약, 단체이용가능, 포장청주 가경동맛집 금성식당은 시외버스터미널에서석곡사거리...","[0 예약, 단체이용가능, 포장청주 가경동맛집 금성식당은 시외버스터미널에서석곡사거리...",[후식으로는 아이스크림과 커피가 있어요 추가 반찬은 셀프바를 이용하면 되고라면사리같...,"[위치 충북 청주시 흥덕구 강서로 77, 0 예약, 단체이용가능, 포장청주 가경동맛..."
8,청주 가경동: 식당 시설된상가/ 양도양수 조건 탄탄한 단골손님...,​가경동 1층 식당시설된매출 우수한 상가 어때?안녕하세요 청주새로이 부동산입니다....,2025-06-12,​가경동 1층 식당시설된매출 우수한 상가 어때?안녕하세요 청주새로이 부동산입니다....,[대로변 1층 상가로 자동 광고효과 전면 넓은 통유리로 노출성까지 겸비된 상가 거주...,[가경동 1층 식당시설된매출 우수한 상가 어때?안녕하세요 청주새로이 부동산입니다.오...,"[신규 창업을 고민하는 분들, 이전 계획이 있으신 분초기 비용 절감할 수 있는 이 ...",[가경동 1층 식당시설된매출 우수한 상가 어때?안녕하세요 청주새로이 부동산입니다.오...
9,청주 가경동 초밥 뷔페 쿠우쿠우 모임하기 좋은 식당 추천 :),"​안녕하세요 😚오늘은 청주에 있는 스시 뷔페, 가경동 초밥 뷔페인 쿠우쿠우 청주점을...",2025-07-04,"​안녕하세요 . 오늘은 청주에 있는 스시 뷔페, 가경동 초밥 뷔페인 쿠우쿠우 청주점...","[오늘은 청주에 있는 스시 뷔페, 가경동 초밥 뷔페인 쿠우쿠우 청주점을소개해볼께요,...","[쿠우쿠우 청주점 충북 청주시 흥덕구 풍산로 15 2층 가경동,대우메가폴리스 롯데마...","[쿠우쿠우 청주점 충북 청주시 흥덕구 풍산로 15 2층 가경동,대우메가폴리스 롯데마...","[쿠우쿠우 청주점 충북 청주시 흥덕구 풍산로 15 2층 가경동,대우메가폴리스 롯데마..."


In [16]:
cheongju_final.to_excel("./crawled_data/merged_cheongju_analyzed.xlsx", index=False)